# Elasticsearch

このノートブックでは、Elasticsearchを使って、テキストをインデックスし、検索する方法を紹介します。

ノートブックを動かすためには、Elasticsearchインスタンスを立ち上げている必要があります。ノートブックにはColabで動かす手順を書いていますが、そうでない場合は以下のステップを実行する必要があります。

1. マシン上のelasticsearch-X.Y.Z/binフォルダへ移動
2. `./elasticsearch`を実行


## 準備

### パッケージのインストール

In [1]:
!pip install -q elasticsearch==7.9.1

     |████████████████████████████████| 219 kB 7.2 MB/s 


### インポート

In [2]:
import time
from datetime import datetime

from elasticsearch import Elasticsearch

### データセットのダウンロード

データセットとしては「CMU Book Summary Dataset」を使います。このデータセットは、Wikipediaから16,559冊の本のあらすじを抽出して作成されています。タブ区切りで、以下の情報が格納されています。

1. Wikipedia article ID
2. Freebase ID
3. Book title
4. Author
5. Publication date
6. Book genres (Freebase ID:name tuples)
7. Plot summary

データセットをダウンロードして展開します。

In [3]:
!wget https://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz
!tar xvfz booksummaries.tar.gz

--2021-09-26 22:00:33--  https://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16795330 (16M) [application/x-gzip]
Saving to: ‘booksummaries.tar.gz’

booksummaries.tar.g 100%[===================>]  16.02M   200KB/s    in 75s     

2021-09-26 22:01:49 (218 KB/s) - ‘booksummaries.tar.gz’ saved [16795330/16795330]

booksummaries/
booksummaries/README
booksummaries/booksummaries.txt


### ElasticSearchインスタンスのセットアップ

In [4]:
%%bash
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


インスタンスをデーモンのプロセスとして立ち上げます。

In [5]:
%%bash --bg
sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


インスタンスがスタートするまで待つため、何秒かスリープさせましょう。

In [6]:
time.sleep(20)

インスタンスが立ち上がったら、grepでプロセスリストから`elasticsearch`を検索して、利用可能な状態になっているか確認します。

In [7]:
%%bash
ps -ef | grep elasticsearch

root         124     122  0 22:02 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       125     124 97 22:02 ?        00:00:20 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-3871509100367782608 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

クラスタに関する情報を検索するため、エンドポイントにクエリを投げてみましょう。

In [8]:
%%bash
curl -sX GET "localhost:9200/"

{
  "name" : "e1c7fab4f5b5",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "grvYFXa5SfqKUB9UnKf-Mw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


## Elasticsearchの利用

Elasticsearchのインスタンスが立ち上がったので、使ってみましょう。まずは、既に`myindex`という名のインデックスが存在する場合は削除します。

In [9]:
# 既にインデックスが存在する場合は削除
es = Elasticsearch([{"host": "localhost", "port": 9200}])
if es.indices.exists(index="myindex"):
    es.indices.delete(
        index="myindex", ignore=[400, 404]
    )

次に、アップロードしたデータセットを一行ずつ読み取り、インデックスに登録します。`doc`の内容を登録しています。

In [10]:
# 500件だけ使う
path = "booksummaries/booksummaries.txt"  # Add your path.
count = 1
for line in open(path):
    fields = line.split("\t")
    doc = {
        "id": fields[0],
        "title": fields[2],
        "author": fields[3],
        "summary": fields[6],
    }

    res = es.index(index="myindex", id=fields[0], body=doc)
    count = count + 1
    if count % 100 == 0:
        print("indexed 100 documents")
    if count == 501:
        break

indexed 100 documents
indexed 100 documents
indexed 100 documents
indexed 100 documents
indexed 100 documents


インデックスの大きさを確認してみましょう。

In [11]:
res = es.search(index="myindex", body={"query": {"match_all": {}}})
print("Your index has %d entries" % res["hits"]["total"]["value"])

Your index has 425 entries


テストクエリを投げてみます。このクエリでは、`summary`フィールドに対して`animal`が含まれているかを全文検索しています。

In [12]:
res = es.search(index="myindex", body={"query": {"match": {"summary": "animal"}}})
print("Your search returned %d results." % res["hits"]["total"]["value"])

Your search returned 16 results.


クエリした結果を表示してみましょう。タイトルとサマリの先頭100文字だけ表示しています。

In [13]:
print(res["hits"]["hits"][2]["_source"]["title"])
print(res["hits"]["hits"][2]["_source"]["summary"][:100])

Dead Air
 The first person narrative begins on 11 September 2001, and Banks uses the protagonist's conversati


最後に、本の検索をしてみましょう。「STOP」と入力するまで検索し続けることができます。

In [14]:
# match query considers both exact matches, and fuzzy matches and works as a OR query.
# match_phrase looks for exact matches.
while True:
    query = input("Enter your search query: ")
    if query == "STOP":
        break
    res = es.search(
        index="myindex", body={"query": {"match_phrase": {"summary": query}}}
    )
    print("Your search returned %d results:" % res["hits"]["total"]["value"])
    for hit in res["hits"]["hits"]:
        print(hit["_source"]["title"])
        # to get a snippet 100 characters before and after the match
        loc = hit["_source"]["summary"].lower().index(query)
        print(hit["_source"]["summary"][:100])
        print(hit["_source"]["summary"][loc - 100 : loc + 100])

Enter your search query: countess
Your search returned 7 results:
All's Well That Ends Well
 Helena, the orphan daughter of a famous physician, is the ward of the Countess of Rousillon, and ho

The Last Man
 Mary Shelley states in the introduction that in 1818 she discovered, in the Sibyl's cave near Naple
ng leaves the throne, the monarchy come to an end and a republic is created. When the king dies the Countess attempts to raise their son, Adrian, to reclaim the throne, but Adrian opposes his mother a
The Luck of Barry Lyndon
 Redmond Barry of Bally Barry, born to a genteel but ruined Irish family, fancies himself a gentlema
chy, where they win considerable sums of money and Redmond cleverly sets up a plan to marry a young countess of some means. Again, fortune turns against him, and a series of circumstances undermines h
Carmilla
 The story is presented by Le Fanu as part of the casebook of Dr Hesselius, whose departures from me
ily heirloom restored portraits arrives at the castle,

今回は、キーワードベースでの検索をしましたが、Elasticsearchはベクトルを使って類似文書を検索する機能も備えています。その機能を使うことで、たとえば、BERTのようなモデルを使って、文書とクエリを固定長のベクトルに変換し、類似文書を高速に検索できます。興味のある方は、キーワード検索との違いを理解するために試してみるとよいでしょう。

ちなみに、TensorFlow-IOを使うことで、Elasticsearchのデータを使ってTensorFlowのモデルを学習することもできます。興味のある方は、以下のノートブックを参照してください。

- [Streaming structured data from Elasticsearch using Tensorflow-IO](https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/elasticsearch.ipynb#scrollTo=ILyohKWQ_XQS)